In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('explode_spark') \
    .getOrCreate()

22/06/13 11:00:34 WARN Utils: Your hostname, LAPTOP-OJOMANDE resolves to a loopback address: 127.0.1.1; using 172.16.0.52 instead (on interface eth0)
22/06/13 11:00:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/13 11:00:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
df_tracks = spark.read.option("header",True).csv('/home/febridev/tracks.csv')

In [15]:
df_tracks = df_tracks.select(df_tracks.id, df_tracks.name,f.translate(f.col("id_artists"),"[\\[]\\'']", "").alias("id_artists"))
df_tracks.printSchema()
df_tracks.show(truncate=False)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- id_artists: string (nullable = true)

+----------------------+-----------------------------------+----------------------+
|id                    |name                               |id_artists            |
+----------------------+-----------------------------------+----------------------+
|35iwgR4jXetI318WEWsa1Q|Carve                              |45tIt06XoI0Iio4LBEVpls|
|021ht4sdgPcrDgSk7JTbKY|Capítulo 2.16 - Banquero Anarquista|14jtPCOoNZwquk5wd9DxrY|
|07A5yehtSnoedViJAZkNnc|Vivo para Quererte - Remasterizado |5LiOoJbxVSAMkBS2fUm3X2|
|08FmqUhxtyLTn6pAh6bk45|El Prisionero - Remasterizado      |5LiOoJbxVSAMkBS2fUm3X2|
|08y9GfoqCWfOGsKdwojr5e|Lady of the Evening                |3BiJGZsyX9sJchTqcSA7Su|
|0BRXJHRNGQ3W4v9frnSfhu|Ave Maria                          |3BiJGZsyX9sJchTqcSA7Su|
|0Dd9ImXtAtGwsmsAD69KZT|La Butte Rouge                     |2nuMRGzeJ5jJEKlfS7rZ0W|
|0IA0Hju8CAgYfV1hwhidBH|La Java           

In [16]:
df_tracks_cl = df_tracks.select(df_tracks.id, df_tracks.name, f.split(df_tracks.id_artists,",").alias("artists_array"))
df_tracks_cl.printSchema()
df_tracks_cl.show(truncate=False)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- artists_array: array (nullable = true)
 |    |-- element: string (containsNull = true)

+----------------------+-----------------------------------+------------------------+
|id                    |name                               |artists_array           |
+----------------------+-----------------------------------+------------------------+
|35iwgR4jXetI318WEWsa1Q|Carve                              |[45tIt06XoI0Iio4LBEVpls]|
|021ht4sdgPcrDgSk7JTbKY|Capítulo 2.16 - Banquero Anarquista|[14jtPCOoNZwquk5wd9DxrY]|
|07A5yehtSnoedViJAZkNnc|Vivo para Quererte - Remasterizado |[5LiOoJbxVSAMkBS2fUm3X2]|
|08FmqUhxtyLTn6pAh6bk45|El Prisionero - Remasterizado      |[5LiOoJbxVSAMkBS2fUm3X2]|
|08y9GfoqCWfOGsKdwojr5e|Lady of the Evening                |[3BiJGZsyX9sJchTqcSA7Su]|
|0BRXJHRNGQ3W4v9frnSfhu|Ave Maria                          |[3BiJGZsyX9sJchTqcSA7Su]|
|0Dd9ImXtAtGwsmsAD69KZT|La Butte Rouge                    

In [20]:
df2 = df_tracks_cl.select(df_tracks_cl.id, df_tracks_cl.name, f.explode(df_tracks_cl.artists_array).alias("artists_array"))
df2.show(1000,truncate=False)

+----------------------+---------------------------------------------------------------------------------+-----------------------+
|id                    |name                                                                             |artists_array          |
+----------------------+---------------------------------------------------------------------------------+-----------------------+
|35iwgR4jXetI318WEWsa1Q|Carve                                                                            |45tIt06XoI0Iio4LBEVpls |
|021ht4sdgPcrDgSk7JTbKY|Capítulo 2.16 - Banquero Anarquista                                              |14jtPCOoNZwquk5wd9DxrY |
|07A5yehtSnoedViJAZkNnc|Vivo para Quererte - Remasterizado                                               |5LiOoJbxVSAMkBS2fUm3X2 |
|08FmqUhxtyLTn6pAh6bk45|El Prisionero - Remasterizado                                                    |5LiOoJbxVSAMkBS2fUm3X2 |
|08y9GfoqCWfOGsKdwojr5e|Lady of the Evening                                        

In [18]:
df2.where(df2.id == "2LDVHvhGJ4KRJtOVAKSiY2").show(truncate=False)

+----------------------+-----------------------------------------------------------+-----------------------+
|id                    |name                                                       |col                    |
+----------------------+-----------------------------------------------------------+-----------------------+
|2LDVHvhGJ4KRJtOVAKSiY2|Schlichte Weisen, Op. 76: Mariä Wiegenlied (Mary's Lulluby)|2qpchAUtJNY45rG7auwQgz |
|2LDVHvhGJ4KRJtOVAKSiY2|Schlichte Weisen, Op. 76: Mariä Wiegenlied (Mary's Lulluby)| 0hrVe30cEV1RSrsDMYZ388|
|2LDVHvhGJ4KRJtOVAKSiY2|Schlichte Weisen, Op. 76: Mariä Wiegenlied (Mary's Lulluby)| 7vEPPI71V8dEHtEhPMAxWT|
|2LDVHvhGJ4KRJtOVAKSiY2|Schlichte Weisen, Op. 76: Mariä Wiegenlied (Mary's Lulluby)| 4M2ts3aBI6wIWlmzsDjziq|
+----------------------+-----------------------------------------------------------+-----------------------+



In [21]:
df2.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- artists_array: string (nullable = true)

